In [1]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [1]:
import cv2
import numpy as np

In [3]:
#https://github.com/spmallick/learnopencv/blob/5d9e05bce247e475488a40cfec0de11c006db532/Homography/utils.py

def mouse_handler(event, x, y, flags, data) :
    
    if event == cv2.EVENT_LBUTTONDOWN :
        cv2.circle(data['im'], (x,y),3, (0,0,255), 5, 16)
        cv2.imshow("Image", data['im']);
        if len(data['points']) < 4 :
            data['points'].append([x,y])
        
def get_four_points(im):
    
    # Set up data to send to mouse handler
    data = {}
    data['im'] = im.copy()
    data['points'] = []
    
    #Set the callback function for any mouse event
    cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
    cv2.imshow("Image",im)
    cv2.setMouseCallback("Image", mouse_handler, data)
    while cv2.waitKey(0) != ord('q'):
        pass
    cv2.destroyAllWindows()
    # Convert array to np.array
    points = np.vstack(data['points']).astype(float)
    
    return points

In [4]:
"""
https://github.com/jrosebr1/imutils/blob/master/imutils/convenience.py
"""
def grab_contours(cnts):
    # if the length the contours tuple returned by cv2.findContours
    # is '2' then we are using either OpenCV v2.4, v4-beta, or
    # v4-official
    if len(cnts) == 2:
        cnts = cnts[0]

    # if the length of the contours tuple is '3' then we are using
    # either OpenCV v3, v4-pre, or v4-alpha
    elif len(cnts) == 3:
        cnts = cnts[1]

    # otherwise OpenCV has changed their cv2.findContours return
    # signature yet again and I have no idea WTH is going on
    else:
        raise Exception(("Contours tuple must have length 2 or 3, "
            "otherwise OpenCV changed their cv2.findContours return "
            "signature yet again. Refer to OpenCV's documentation "
            "in that case"))

    # return the actual contours array
    return cnts

TESTE DO OTSU PRA CROP -> HOMOGRAPHY

https://stackoverflow.com/questions/60780831/python-how-to-cut-out-an-area-with-specific-color-from-image-opencv-numpy

In [5]:
cv2.destroyAllWindows()

mega clean == blur no output de canny
mesa branca == blur em src -> otsu no blur

In [10]:
fname = "./judd.webm"
cap = cv2.VideoCapture(fname)
ret, frame = cap.read()

src = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
dst = cv2.Canny(src, 50, 100, None, 3)
blur = cv2.GaussianBlur(src,(5,5),0)
ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

lines = cv2.HoughLinesP(th3, 1, np.pi / 180, 150, None, 0, 0)
for line in lines:
	    x1, y1, x2, y2 = line[0]
	    cv2.line(src, (x1, y1), (x2, y2), (255, 0, 0), 3)
cv2.namedWindow('Result Image', cv2.WINDOW_NORMAL)
cv2.imshow("Result Image", th3)
while cv2.waitKey(0) != ord('q'):
        pass
cv2.destroyAllWindows()


In [30]:
cap = cv2.VideoCapture(fname)

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("eof")
        break

    src = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    blur = cv2.GaussianBlur(src,(5,5),0)


    ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    cnts, _ = cv2.findContours(th3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    c = max(cnts, key = cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)

    cv2.drawContours(frame, c, -1, (255, 0, 0), 3)

    cv2.namedWindow('Result Image', cv2.WINDOW_NORMAL)
    cv2.imshow("Result Image", frame)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
#outv.release(
cv2.destroyAllWindows()

eof


In [ ]:
fname = "./judd.webm"
cap = cv2.VideoCapture(fname)

shot = []

# TODO: Automatic table corner detection
ret, frame = cap.read()
#cv2.imshow("Image", frame)
pts_src = get_four_points(frame)

In [ ]:
size = (500, 1000, 3)
pts_dst = np.array(
               [
                [0,0],
                [size[0] - 1, 0],
                [size[0] - 1, size[1] -1],
                [0, size[1] - 1 ]
                ], dtype=float
               )

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("eof")
        break

    h, status = cv2.findHomography(pts_src, pts_dst)
    im_dst = cv2.warpPerspective(frame, h, size[:2])
    cv2.namedWindow('Image', cv2.WINDOW_NORMAL)
    cv2.imshow("Image", im_dst)
    shot.append(im_dst)
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()

In [87]:
cv2.imwrite("test.jpg", shot[0])

for frame in shot:
    # Smooth
    frame_blur = cv2.GaussianBlur(frame, (0,0), 2)
    frame_hsv = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2HSV)

    # Snooker green color range for judd.webm
    lowerb = np.array([52, 150, 50])
    upperb = np.array([60, 255, 220])
    mask = cv2.inRange(frame_hsv, lowerb, upperb)

    kernel = np.ones((5,5), np.uint8)
    mask_closing = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel) # Cleans inner points
    
    # Applies the mask to the original frame
    _,mask_inv = cv2.threshold(mask_closing,5,255,cv2.THRESH_BINARY_INV)
    masked_img = cv2.bitwise_and(frame,frame, mask=mask_inv)

    ctrs, hierarchy = cv2.findContours(mask_inv, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    ctrs = [ctr for ctr in ctrs
                        if 300 < cv2.contourArea(ctr) < 1000
                        #and cv2.minAreaRect(ctr)[1][0]*0.28 < cv2.minAreaRect(ctr)[1][1]
                        #and cv2.minAreaRect(ctr)[1][1]*0.28 < cv2.minAreaRect(ctr)[1][0]
            ]
    frame = frame.copy()
    #draw_rectangles(ctrs, frame)
    cv2.drawContours(frame, ctrs, -1, (255, 0, 0), 3)
    cv2.namedWindow('FRAME', cv2.WINDOW_NORMAL)
    cv2.imshow("FRAME", frame)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()


In [78]:
cv2.destroyAllWindows()


In [40]:
roi = cv2.selectROI("tracker", shot[0])
cv2.destroyAllWindows()


In [46]:
tracker = cv2.TrackerMIL_create()
tracker.init(shot[0], roi)
for frame in shot:
    frame = frame.copy()
    _, roi = tracker.update(frame)
    cv2.rectangle(frame, roi, (255, 0, 0), 2, 1)
    cv2.imshow("tracker", frame)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()


error: OpenCV(4.5.3) /build/opencv/src/opencv-4.5.3/modules/video/src/tracking/detail/tracking_feature.cpp:128: error: (-215:Assertion failed) !patchSize.empty() in function 'generateRandomFeature'


In [ ]:
tracker = cv2.TrackerCSRT(shot[0], roi)


In [21]:
trackers = cv2.legacy.MultiTracker_create()


AttributeError: module 'cv2' has no attribute 'MultiTracker'

In [11]:
import cv2
import numpy as np

def getContours(img):
    cnts, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    c = max(cnts, key=cv2.contourArea)
    approx = cv2.approxPolyDP(c, 0.03 * cv2.arcLength(c, True), True)
    return approx, c

def order_points(points):
    center = np.mean(points)
    shifted = points - center
    theta = np.arctan2(shifted[:, 0], shifted[:, 1])
    ind = np.argsort(theta)
    return points[ind]



In [12]:
fname = "./judd.webm"
cap = cv2.VideoCapture(fname)
ret, frame = cap.read()

gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5,5), 1)
canny = cv2.Canny(blur, 50, 150)
kernel = np.ones((5,5), np.uint8)
diler = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)
cv2.imshow("TESTE", diler)
if cv2.waitKey(0) == ord('q'):
    cv2.destroyAllWindows()
pts_src, c = getContours(diler)

cv2.drawContours(frame, c, -1, (255, 0, 0), 3)
cv2.imshow("TESTE", frame)
while True:
    if cv2.waitKey(0) == ord('q'):
        cv2.destroyAllWindows()
        break
size = (500, 1000, 3)

pts_dst = np.array(
               [
                [0,0],
                [size[0] - 1, 0],
                [size[0] - 1, size[1] -1],
                [0, size[1] - 1 ]
                ], dtype=float
               )
pts_src = order_points(pts_src[:, 0])
pts_dst = order_points(pts_dst)
print(pts_src)
print(pts_dst)
h, status = cv2.findHomography(pts_src, pts_dst)
im_dst = cv2.warpPerspective(frame, h, size[:2])
cv2.imshow("IMAGE", im_dst)
while True:
    if cv2.waitKey(0) == ord('q'):
        cv2.destroyAllWindows()
        break

[[ 540  295]
 [ 358  892]
 [1559  896]
 [1365  290]]
[[  0.   0.]
 [  0. 999.]
 [499. 999.]
 [499.   0.]]


In [16]:
pts_src

array([[[ 540,  295]],

       [[ 358,  892]],

       [[1559,  896]],

       [[1365,  290]]], dtype=int32)

In [23]:
from detector_juba.table_contours import get_cloth_mask, get_ball_contours, get_homography, get_cloth_range
import cv2
import numpy as np
video = []

fname = "./judd.webm"
cap = cv2.VideoCapture(fname)
ret, frame = cap.read()

h = get_homography(frame, (600, 1000))
im_dst = cv2.warpPerspective(frame, h, (600, 1000))

hsv = cv2.cvtColor(im_dst, cv2.COLOR_BGR2HSV)
hue, s, v = cv2.split(hsv)
lower, upper = get_cloth_range(hue, s, v)

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("eof")
        break


    im_dst = cv2.warpPerspective(frame, h, (600, 1000))
    warped = im_dst.copy()
    mask = get_cloth_mask(im_dst, lower, upper)

    ctrs = get_ball_contours(mask)

    for c in ctrs:
        x, y, w, z = cv2.boundingRect(c)
        x -= 10
        y -= 10
        w += 15
        z += 15
        cv2.rectangle(im_dst,(x,y),(x+w,y+z),(0,0,255),2)

    masked_img = cv2.bitwise_and(im_dst, im_dst, mask=mask)

    cv2.drawContours(im_dst, ctrs, -1, (255, 0, 0), 3)

    im_h = cv2.hconcat([warped, cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR), masked_img, im_dst])
    video.append(im_h)
    cv2.imshow("Image", im_h)
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()


eof


In [25]:
def saveVideo(frames, size=(640, 480)):
    fourcc = cv2.VideoWriter_fourcc(*'FMP4')
    out = cv2.VideoWriter('output.mp4', fourcc, 30, (2400, 1000))
    for frame in frames:
        out.write(frame)
    out.release()

saveVideo(video)


In [5]:
cv2.destroyAllWindows()
